# Inspecting the content of possibly misclassified samples


After performin the clustering phase we compared the results with a baseline clustering provided by AV labels. 

From this comparison it was clear that there were some malware families which where classified in the same way by both our clustering and the AVs.

At the same time, however, there are groups of samples which result close in our feature space while being cathegorized as belonging to different families by the AVs.

We would like to inspect this samples to better understand why they were classified differently from the AV baseline.

Let's start by importing some useful packages.

In [1]:
%load_ext autoreload
%autoreload 2

from collections import defaultdict, Counter
from utilities import constants
from pprint import pprint
import plotly.offline as ply
import pandas as pd
import numpy as np
import json
import os

In [2]:
config = json.load(open('config.json', 'r'))
uuids_family = json.load(open(os.path.join(constants.dir_d, constants.json_labels), 'r'))
words = json.load(open(os.path.join(constants.dir_d, constants.json_words), 'r'))
ply.init_notebook_mode(connected=True)

Next we load the labels and clustering results files

In [3]:
labels = json.load(open('data/labels.json', 'r'))
inv_labels = json.load(open('data/inverted_labels.json', 'r'))

clustering = json.load(open('data/d_clusterings/clustering_hdbscan_cosine_1209.json', 'r'))

In [4]:
clust_compositions = {i: Counter() for i in sorted(set(clustering.values()))}

for i in clustering:
    clust_compositions[clustering[i]][labels[i]] += 1

for clu in sorted(clust_compositions.keys()):
    print('Cluster {}:'.format(clu))
    print(clust_compositions[clu].most_common())
    print()

Cluster -1:
[('eorezo', 56), ('bladabindi', 54), ('flystudio', 15), ('gepys', 2), ('mydoom', 2), ('lamer', 2)]

Cluster 0:
[('neshta', 224)]

Cluster 1:
[('mydoom', 144), ('eorezo', 3), ('flystudio', 1)]

Cluster 2:
[('lamer', 131), ('eorezo', 1)]

Cluster 3:
[('eorezo', 108), ('bladabindi', 9)]

Cluster 4:
[('eorezo', 87), ('gepys', 74), ('flystudio', 62), ('bladabindi', 47), ('neshta', 6), ('lamer', 3)]

Cluster 5:
[('bladabindi', 159), ('eorezo', 11), ('flystudio', 8)]



In [5]:
inverted_clustering = defaultdict(list)
for i in clustering:
    inverted_clustering[clustering[i]].append(i)

Let's isolate the noise cluster, i.e. the samples which the algorithm was unable to fit in a cluster.

In [6]:
noise = inverted_clustering[-1]

This cluster seems composed primarily by samples of the Eorezo and Bladabindi families.

In [7]:
noise_e = []
noise_b = []

for uuid in noise:
    if uuids_family[uuid] == 'eorezo':
        noise_e.append(uuid)
    elif uuids_family[uuid] == 'bladabindi':
        noise_b.append(uuid)

noise_e = sorted(noise_e)
noise_b = sorted(noise_b)

pprint(noise_e[:5])
pprint(noise_b[:5])

['00b47107-48c5-47aa-8673-4e15b61b1846',
 '02f02dd2-1a8d-4831-94a0-e7476f30c73d',
 '0ae9bfe9-5d65-474d-908b-382a1e6dcdc6',
 '0b1d113a-e6b3-48c4-b3fe-a5548c0d0337',
 '0f9ff148-9084-4a08-a82e-009e7d92f1cf']
['0248cc75-288b-43c0-94c8-e5aa466fcf5c',
 '0423c18b-8250-4299-a947-0bbd707b0d67',
 '0788d49d-4809-40a7-a334-bd56e41b1333',
 '0e8a702a-5a33-4de0-8de1-c3fcbeae6e48',
 '0fc9cf5a-1f75-4429-af1c-033d4821e096']


Similarly for cluster number 4

In [8]:
clus4 = inverted_clustering[4]

This time it seems this cluster should have been populated primarily by the Flystudio or the Gepys family. However a large number of samples from both Eorezo and Bladabindi are included in this cluster.

In [9]:
clus4_e = []
clus4_b = []
clus4_g = []
clus4_f = []

for uuid in clus4:
    if uuids_family[uuid] == 'eorezo':
        clus4_e.append(uuid)
    elif uuids_family[uuid] == 'bladabindi':
        clus4_b.append(uuid)
    elif uuids_family[uuid] == 'gepys':
        clus4_g.append(uuid)
    elif uuids_family[uuid] == 'flystudio':
        clus4_f.append(uuid)


clus4_e = sorted(clus4_e)
clus4_b = sorted(clus4_b)
clus4_g = sorted(clus4_g)
clus4_f = sorted(clus4_f)

pprint(clus4_e[:5])
pprint(clus4_b[:5])
pprint(clus4_g[:5])
pprint(noise_b[:5])

['006953d6-8a8a-4938-bda1-987733b970cd',
 '00a2434b-6852-4e50-8b5c-bf5cc1da5ec1',
 '0413eaa7-5431-4d2b-9510-0781508eae02',
 '0842d66d-8c15-405a-88ba-10a916bac7ea',
 '09b85bd8-d4ea-4d64-8363-facad113e7b4']
['059fed1d-1577-4bd3-a380-bdf3adb278e8',
 '071aa948-8a20-423d-84a8-17d312cd5f28',
 '1c410f27-6b28-4ead-b2d1-53fcf3132394',
 '1f1ab0e7-d53c-4a88-9ac1-c58197d42302',
 '283cd153-36f2-4a05-ad78-d9df63d202a3']
['00b4a2aa-3216-435a-80b2-1db8b9c186ca',
 '062abbb2-324e-49b5-952d-a11716763e2f',
 '0baaa6fa-ef83-4632-8786-03f77ef83920',
 '201194a3-ecc7-44d4-9cee-337b1df9bcfe',
 '27738e13-4be3-4e3d-b2c9-de5b2d6d1e8d']
['0248cc75-288b-43c0-94c8-e5aa466fcf5c',
 '0423c18b-8250-4299-a947-0bbd707b0d67',
 '0788d49d-4809-40a7-a334-bd56e41b1333',
 '0e8a702a-5a33-4de0-8de1-c3fcbeae6e48',
 '0fc9cf5a-1f75-4429-af1c-033d4821e096']
